In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Custom Dataset class
class TORCSDataset(Dataset):
    def __init__(self, csv_file, sensor_cols, continuous_cols, discrete_cols):
        self.data = pd.read_csv(csv_file)
        self.sensor_cols = [col for col in self.data.columns if col in sensor_cols]
        # self.sensor_cols = sensor_cols
        self.continuous_cols = [col for col in self.data.columns if col in continuous_cols]
        # self.continuous_cols = continuous_cols
        self.discrete_cols = [col for col in self.data.columns if col in discrete_cols]
        # self.discrete_cols = discrete_cols
        self.features = torch.tensor(self.data[self.sensor_cols].values, dtype=torch.float32)
        self.continuous_labels = torch.tensor(self.data[self.continuous_cols].values, dtype=torch.float32)
        self.gear_labels = torch.tensor(self.data['Gear'].values, dtype=torch.long)
        self.clutch_labels = torch.tensor(self.data['Clutch'].values, dtype=torch.long)
        # Adjust Gear/Clutch to zero-based indexing
        self.gear_labels = self.gear_labels - self.gear_labels.min()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.features[idx], self.continuous_labels[idx], 
                self.gear_labels[idx], self.clutch_labels[idx])

In [16]:
# Neural Network Model
class TORCSNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_gear_classes, num_clutch_classes):
        super(TORCSNet, self).__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
            # To add a third hidden layer, uncomment:
            # nn.Linear(hidden_size, hidden_size),
            # nn.ReLU()
        )
        self.continuous_head = nn.Linear(hidden_size, 3)  # Acceleration, Braking, Steering
        self.gear_head = nn.Linear(hidden_size, num_gear_classes)
        self.clutch_head = nn.Linear(hidden_size, num_clutch_classes)

    def forward(self, x):
        shared_features = self.shared(x)
        continuous_out = self.continuous_head(shared_features)
        accel_brake = torch.sigmoid(continuous_out[:, :2])  # [0, 1]
        steering = torch.tanh(continuous_out[:, 2])         # [-1, 1]
        continuous_out = torch.cat([accel_brake, steering.unsqueeze(1)], dim=1)
        gear_out = self.gear_head(shared_features)
        clutch_out = self.clutch_head(shared_features)
        return continuous_out, gear_out, clutch_out

In [ ]:
# Training function
def train_model(model, train_loader, val_loader, num_epochs, device):
    criterion_cont = nn.MSELoss()
    criterion_disc = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for features, cont_labels, gear_labels, clutch_labels in train_loader:
            features = features.to(device)
            cont_labels = cont_labels.to(device)
            gear_labels = gear_labels.to(device)
            clutch_labels = clutch_labels.to(device)
            
            optimizer.zero_grad()
            cont_out, gear_out, clutch_out = model(features)
            
            loss_cont = criterion_cont(cont_out, cont_labels)
            loss_gear = criterion_disc(gear_out, gear_labels)
            loss_clutch = criterion_disc(clutch_out, clutch_labels)
            loss = loss_cont + loss_gear + loss_clutch
            
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * features.size(0)
        
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        gear_correct = 0
        clutch_correct = 0
        total = 0
        with torch.no_grad():
            for features, cont_labels, gear_labels, clutch_labels in val_loader:
                features = features.to(device)
                cont_labels = cont_labels.to(device)
                gear_labels = gear_labels.to(device)
                clutch_labels = clutch_labels.to(device)
                
                cont_out, gear_out, clutch_out = model(features)
                loss_cont = criterion_cont(cont_out, cont_labels)
                loss_gear = criterion_disc(gear_out, gear_labels)
                loss_clutch = criterion_disc(clutch_out, clutch_labels)
                loss = loss_cont + loss_gear + loss_clutch
                val_loss += loss.item() * features.size(0)
                
                # Accuracy for Gear/Clutch
                _, gear_pred = torch.max(gear_out, 1)
                _, clutch_pred = torch.max(clutch_out, 1)
                gear_correct += (gear_pred == gear_labels).sum().item()
                clutch_correct += (clutch_pred == clutch_labels).sum().item()
                total += gear_labels.size(0)
        
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        gear_acc = gear_correct / total
        clutch_acc = clutch_correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, Gear Acc: {gear_acc:.4f}, Clutch Acc: {clutch_acc:.4f}')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pt')
    
    # Plot losses
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.savefig('loss_plot.png')
    plt.show()

    return train_losses, val_losses


In [33]:
import pandas as pd

# Check train_data.csv
train_data = pd.read_csv('../data_generation/data/dirt-2/train_data.csv')
print("Columns in train_data.csv:")
print(train_data.columns.tolist())
print("Number of columns:", len(train_data.columns))

# Check val_data.csv
val_data = pd.read_csv('../data_generation/data/dirt-2/val_data.csv')
print("\nColumns in val_data.csv:")
print(val_data.columns.tolist())
print("Number of columns:", len(val_data.columns))

Columns in train_data.csv:
['Angle', ' CurrentLapTime', ' Damage', ' DistanceFromStart', ' DistanceCovered', ' FuelLevel', ' LastLapTime', ' Opponent_1', 'RacePosition', ' RPM', ' SpeedX', ' SpeedY', ' SpeedZ', ' Track_1', 'Track_2', 'Track_3', 'Track_4', 'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9', 'Track_10', 'Track_11', 'Track_12', 'Track_13', 'Track_14', 'Track_15', 'Track_16', 'Track_17', 'Track_18', 'Track_19', 'TrackPosition', ' WheelSpinVelocity_1', 'WheelSpinVelocity_2', 'WheelSpinVelocity_3', 'WheelSpinVelocity_4', 'Z', ' Acceleration', 'Braking', 'Clutch', 'Gear', 'Steering']
Number of columns: 43

Columns in val_data.csv:
['Angle', ' CurrentLapTime', ' Damage', ' DistanceFromStart', ' DistanceCovered', ' FuelLevel', ' LastLapTime', ' Opponent_1', 'RacePosition', ' RPM', ' SpeedX', ' SpeedY', ' SpeedZ', ' Track_1', 'Track_2', 'Track_3', 'Track_4', 'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9', 'Track_10', 'Track_11', 'Track_12', 'Track_13', 'Track_14', 'Tra

In [ ]:
# Define columns
sensor_cols = ['Angle', ' CurrentLapTime', ' Damage', ' DistanceFromStart', ' DistanceCovered', 
                ' FuelLevel', ' LastLapTime', 'RacePosition', ' RPM', 
               ' SpeedX', ' SpeedY', ' SpeedZ', ' Track_1', 'Track_2', 'Track_3', 
               'Track_4', 'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9', 
               'Track_10', 'Track_11', 'Track_12', 'Track_13', 'Track_14', 'Track_15', 
               'Track_16', 'Track_17', 'Track_18', 'Track_19', 'TrackPosition', 
                ' WheelSpinVelocity_1', 'WheelSpinVelocity_2', 'WheelSpinVelocity_3', 
               'WheelSpinVelocity_4', 'Z']
continuous_cols = [' Acceleration', 'Braking', 'Steering']
discrete_cols = ['Gear', 'Clutch']
    
# Load datasets
train_dataset = TORCSDataset('../data_generation/data/dirt-2/train_data.csv', sensor_cols, continuous_cols, discrete_cols)
val_dataset = TORCSDataset('../data_generation/data/dirt-2/val_data.csv', sensor_cols, continuous_cols, discrete_cols)
    
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)


In [43]:
print(len(sensor_cols))
# print(continuous_cols)

print(len(train_dataset.sensor_cols))

train_dataset.sensor_cols
# val_dataset.sensor_cols

37
36


['Angle',
 ' CurrentLapTime',
 ' Damage',
 ' DistanceCovered',
 ' FuelLevel',
 ' LastLapTime',
 'RacePosition',
 ' RPM',
 ' SpeedX',
 ' SpeedY',
 ' SpeedZ',
 ' Track_1',
 'Track_2',
 'Track_3',
 'Track_4',
 'Track_5',
 'Track_6',
 'Track_7',
 'Track_8',
 'Track_9',
 'Track_10',
 'Track_11',
 'Track_12',
 'Track_13',
 'Track_14',
 'Track_15',
 'Track_16',
 'Track_17',
 'Track_18',
 'Track_19',
 'TrackPosition',
 ' WheelSpinVelocity_1',
 'WheelSpinVelocity_2',
 'WheelSpinVelocity_3',
 'WheelSpinVelocity_4',
 'Z']

In [37]:
    # Model parameters
input_size = len(sensor_cols)
print(input_size)
hidden_size = 128
num_gear_classes = len(train_dataset.data['Gear'].unique())
num_clutch_classes = len(train_dataset.data['Clutch'].unique())
    
# Initialize model
device = torch.device('cpu')
model = TORCSNet(input_size, hidden_size, num_gear_classes, num_clutch_classes).to(device)

37


In [38]:
# Main execution
if __name__ == "__main__":
    # # Define columns
    # sensor_cols = ['Angle', 'CurrentLapTime', 'Damage', 'DistanceFromStart', 'DistanceCovered', 
    #                'FuelLevel', 'gear_drop', 'LastLapTime', 'RacePosition', 'RPM', 
    #                'SpeedX', 'SpeedY', 'SpeedZ', 'Track_1', 'Track_2', 'Track_3', 
    #                'Track_4', 'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9', 
    #                'Track_10', 'Track_11', 'Track_12', 'Track_13', 'Track_14', 'Track_15', 
    #                'Track_16', 'Track_17', 'Track_18', 'Track_19', 'TrackPosition', 
    #                'WheelSpinVelocity_1', 'WheelSpinVelocity_2', 'WheelSpinVelocity_3', 
    #                'WheelSpinVelocity_4', 'Z']
    # continuous_cols = ['Acceleration', 'Braking', 'Steering']
    # discrete_cols = ['Gear', 'Clutch']
    
    # # Load datasets
    # train_dataset = TORCSDataset('../data_generation/data/dirt-2/train_data.csv', sensor_cols, continuous_cols, discrete_cols)
    # val_dataset = TORCSDataset('../data_generation/data/dirt-2/val_data.csv', sensor_cols, continuous_cols, discrete_cols)
    
    # train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
    

    
    # Train and plot
    train_losses, val_losses = train_model(model, train_loader, val_loader, num_epochs=50, device=device)
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pt'))
    print("Training complete. Best model saved as 'best_model.pt'. Loss plot saved as 'loss_plot.png'.")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x36 and 37x128)